parsinlu only -> 1 epoch 1e-5 lr 0.7800


faq data -> 1 epoch 1e-5 lr 0.8172

In [1]:
!rm -rf sample_data

# pip installs

In [2]:
! pip install parsivar -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 113.9 MB/s eta 0:00:00


# imports

In [2]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from parsivar import Normalizer
import re
from datasets import load_dataset, concatenate_datasets, DatasetDict, Dataset
from sentence_transformers import SentenceTransformer, InputExample, losses, models
from sentence_transformers import evaluation, SentencesDataset, SentenceTransformerTrainer


In [3]:
import os
os.environ["WANDB_DISABLED"] = "true"

# fine-tune with parsinlu dataset

## load and process parsinlu dataset

In [4]:
!mkdir queries

mkdir: cannot create directory ‘queries’: File exists


In [5]:
! wget https://raw.githubusercontent.com/persiannlp/parsinlu/refs/heads/master/data/qqp/train.jsonl -O queries/train.jsonl -q
! wget https://raw.githubusercontent.com/persiannlp/parsinlu/refs/heads/master/data/qqp/dev.jsonl -O queries/dev.jsonl -q
! wget https://raw.githubusercontent.com/persiannlp/parsinlu/refs/heads/master/data/qqp/test.jsonl -O queries/test.jsonl -q

In [20]:

data_files = {
    'train': 'queries/train.jsonl',
    'validation': 'queries/dev.jsonl',
    'test': 'queries/test.jsonl'
}
ds = load_dataset('json', data_files=data_files)

In [21]:
ds

DatasetDict({
    train: Dataset({
        features: ['q1', 'q2', 'label', 'category'],
        num_rows: 1830
    })
    validation: Dataset({
        features: ['q1', 'q2', 'label', 'category'],
        num_rows: 898
    })
    test: Dataset({
        features: ['q1', 'q2', 'label', 'category'],
        num_rows: 1916
    })
})

In [22]:
normalizer = Normalizer()
def normalize_batch(batch):
    batch["q1"] = normalizer.normalize(batch["q1"])
    batch["q2"] = normalizer.normalize(batch["q2"])
    return batch

ds = ds.map(normalize_batch)

Map:   0%|          | 0/1830 [00:00<?, ? examples/s]

Map:   0%|          | 0/1916 [00:00<?, ? examples/s]

In [23]:
def convert_to_input_examples(split):
    examples = []
    for item in ds[split]:
        examples.append(InputExample(texts=[item["q1"], item["q2"]]))
    return examples

train_examples = convert_to_input_examples("train")
valid_examples = convert_to_input_examples("validation")


## fine-tuning

In [24]:
model = SentenceTransformer("PartAI/Tooka-SBERT-V2-Large")
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=32)
train_loss = losses.MultipleNegativesRankingLoss(model)
evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(valid_examples, name="sts-dev")


In [ ]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=1,
    optimizer_params={'lr': 1e-5},
    warmup_steps=int(len(train_dataloader) * 0.1),
    evaluation_steps=1000,
    output_path="./tooka-sbert-finetuned"
)

# finetune on farsi bank faq data

In [39]:
data_path = 'farsi-bank-faqs.jsonl'

In [40]:
df = pd.read_json(data_path, lines=True)
ds = Dataset.from_pandas(df)


In [41]:
normalizer = Normalizer()
def normalize_batch(batch):
    batch["Q"] = normalizer.normalize(batch["Q"])
    batch["A"] = normalizer.normalize(batch["A"])
    return batch

ds = ds.map(normalize_batch)

Map:   0%|          | 0/1756 [00:00<?, ? examples/s]

In [42]:
ds

Dataset({
    features: ['id', 'Q', 'A', 'class'],
    num_rows: 1756
})

In [43]:
train_examples = [InputExample(texts=[row["Q"], row["A"]]) for row in ds]


In [44]:
model = SentenceTransformer("PartAI/Tooka-SBERT-V2-Large")
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=8)
train_loss = losses.MultipleNegativesRankingLoss(model)

In [45]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    optimizer_params={'lr': 1e-5},
    warmup_steps=int(len(train_dataloader) * 0.1),
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss


# test on target data

In [46]:
sheet_path = '/content/Ka-ChatBot_BenchMark.xlsx'

In [47]:
faq_df = pd.read_excel(sheet_path, sheet_name="faq")
samples_df = pd.read_excel(sheet_path, sheet_name="samples")

In [48]:
faq = faq_df['faq'].tolist()
samples = samples_df['sample'].tolist()
labels = torch.tensor(samples_df['gt_idx'].tolist())

In [49]:
faq_emb = model.encode(faq)
samples_emb = model.encode(samples)
similarities = model.similarity(faq_emb, samples_emb)

predict_indices = torch.max(similarities, axis=0)[1]
predicts = predict_indices + 1

acc = (predicts == labels).sum() / labels.shape[0]
acc

tensor(0.8172)

In [50]:
resultdf = pd.read_csv('results.csv')
resultdf['tooka-sbert-v2-large-finetuned'] = predicts
resultdf.to_csv('newresults.csv', index=False)